In [71]:
import itertools
import logging
import time
import numpy as np

from qiskit.circuit.library import EfficientSU2
from qiskit import Aer
# from qiskit.validation.base import BaseModel, BaseSchema, ObjSchema, bind_schema, Obj
from qiskit.aqua import QuantumInstance
# from qiskit.aqua.operators import Z2Symmetries, WeightedPauliOperator
from qiskit.aqua.algorithms import VQE, NumPyMinimumEigensolver
# from qiskit.aqua.components.optimizers import COBYLA
from qiskit.chemistry.fermionic_operator import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry.core import Hamiltonian, TransformationType, QubitMappingType
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

from pyscf import gto, scf, ao2mo, fci, cc, mp, hci
from pyscf.hci import SelectedCI 
from pyscf.fci import cistring
import numpy as np
import itertools
from collections import Counter
import operator
import matplotlib.pyplot as plt
import pyscf
import h5py

In [72]:
# logging.basicConfig(level=logging.DEBUG)

In [97]:
def convert_t1_to_blocks(t1_ind_list, nelec, norb):
    t1_list = []
    for t1_amp_ind in t1_ind_list:
        t1_list.append([t1_amp_ind[0], t1_amp_ind[1] + int(nelec/2)])
        t1_list.append([t1_amp_ind[0]+norb, t1_amp_ind[1] + norb + int(nelec/2)])
    return t1_list

def convert_t2_to_blocks(t2_ind_list, nelec, norb):
    t2_list = []
    for t2_amp_ind in t2_ind_list:
        t2_list.append([t2_amp_ind[0], t2_amp_ind[2] + int(nelec/2), t2_amp_ind[1]+norb, t2_amp_ind[3] + norb + int(nelec/2)])
    return t2_list

def convert_t3_to_blocks(t3_ind_list, nelec, norb):
    t3_list = []
    for t3_amp_ind in t3_ind_list:
        t3_list.append([t3_amp_ind[0], t3_amp_ind[3] + int(nelec/2),
                        t3_amp_ind[1]+norb, t3_amp_ind[4] + norb + int(nelec/2),
                        t3_amp_ind[2]+norb, t3_amp_ind[5] + norb + int(nelec/2)
                       ])
    return t3_list

def print_sorted_dict(dict_to_print):
    sorted_dict = dict( sorted(dict_to_print.items(), key=lambda kv: np.max(np.abs(kv[1])), reverse=True))
    for key in sorted_dict.keys():
        print(key, sorted_dict[key])


def filter_dets_from_dict(d, thresh=1e-08):
    filtered_dict = dict()
    for key in d.keys():
        if np.max(np.abs(d[key])) > thresh:

            filtered_dict[key] = d[key]
        # else:
        #     print(key, np.max(np.abs(d[key])))
    # print("filtered", filtered_dict)
    return filtered_dict  

def filter_out_excitations_from_det_dict(dict_to_filter, order_of_exc, nelec):
    filtered_dict = dict()
    nocc = int(nelec/2)
    for key in dict_to_filter.keys():
        n_el_occ = 0
        for x in key[0:nocc]:
            n_el_occ += int(x)
        # print("n_el_occ", n_el_occ, key)
        if nelec - n_el_occ == order_of_exc:
            filtered_dict[key] = dict_to_filter[key]
    return filtered_dict

def generate_triple_exc_dets(nelec, norb, occ_array):

    det_vs_amp_ind_dict = dict()

    for exc_from_i in range(int(nelec/2)):
        for exc_from_j in range(int(nelec/2)):
            for exc_from_k in range(int(nelec/2)):
                for exc_to_a in range(norb-int(nelec/2)):
                    for exc_to_b in range(norb-int(nelec/2)):
                        for exc_to_c in range(norb-int(nelec/2)):

                            ini_occ = occ_array.copy()
                            ini_occ[exc_from_i] = ini_occ[exc_from_i] - 1
                            ini_occ[exc_from_j] = ini_occ[exc_from_j] - 1
                            ini_occ[exc_from_k] = ini_occ[exc_from_k] - 1
                            
                            ini_occ[exc_to_a + int(nelec/2)] = ini_occ[exc_to_a + int(nelec/2)] + 1
                            ini_occ[exc_to_b + int(nelec/2)] = ini_occ[exc_to_b + int(nelec/2)] + 1
                            ini_occ[exc_to_c + int(nelec/2)] = ini_occ[exc_to_c + int(nelec/2)] + 1

                            if ini_occ[exc_from_i] in range(0,3) and ini_occ[exc_from_j] in range(0,3) \
                                and ini_occ[exc_from_k] in range(0,3) and ini_occ[exc_to_a + int(nelec/2)] in range(0,3)\
                                    and ini_occ[exc_to_b + int(nelec/2)] in range(0,3) and ini_occ[exc_to_c + int(nelec/2)] in range(0,3):
                                        # valid triple excitation
                                        string = ''
                                        for l in range(np.shape(ini_occ)[0]):
                                            string += str(int(ini_occ[l]))
                                        if string in det_vs_amp_ind_dict.keys():
                                            det_vs_amp_ind_dict[string].append([exc_from_i, exc_from_j, exc_from_k,
                                                                                exc_to_a, exc_to_b, exc_to_c])
                                        else:
                                            det_vs_amp_ind_dict[string] = [[exc_from_i, exc_from_j, exc_from_k,
                                                                            exc_to_a, exc_to_b, exc_to_c]]

    return det_vs_amp_ind_dict

def run_calcs(r, mol, basis, sci_select_cutoff, sci_ci_coeff_cutoff, det_thresh, discarded_amp_thresh, mp2_thresh):
   
    print("r=", r)
    basis = basis
    sci_select_cutoff = sci_select_cutoff
    sci_ci_coeff_cutoff = sci_ci_coeff_cutoff
    det_thresh = det_thresh # filters SCI selected dets (in FCI format there's a lot of zeros)
    discarded_amp_thresh = discarded_amp_thresh # for counting amplitudes filtered out by SCI
    mp2_thresh = mp2_thresh # throw out mp amps smaller than this 

    mol.verbose = 0

    # RHF
    mf = scf.RHF(mol)
    e_hf = mf.kernel()
    h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
    eri = ao2mo.kernel(mol, mf.mo_coeff)
    

    # Full CI
    print("\nFCI")
    cisolver = fci.FCI(mol)
    e_fci, ci = cisolver.kernel(h1, eri, h1.shape[1], mol.nelec, ecore=mol.energy_nuc())
    print("FCI energy", e_fci)
    norb = h1.shape[1]
    nelec = mol.nelectron
    print("nelec", nelec)
    print("norb", norb)
    print("Num Strings: ", cistring.num_strings(norb,nelec/2))

    coeff = list()
    spin_orb_strings = list()
    det_dict = dict()

    for alpha_addr in range(ci.shape[0]):
        for beta_addr in range(ci.shape[0]):
            # In pyscf the lower energy orbitals are to the RIGHT!!! so we have to reverse the strings

            a_str = bin(cistring.addrs2str(norb, int(nelec/2),[alpha_addr])[0])[2:].rjust(norb, '0')[::-1] # converting to strings with occ numbers
            b_str = bin(cistring.addrs2str(norb, int(nelec/2),[beta_addr])[0])[2:].rjust(norb, '0')[::-1]
            
            spin_orb_str = ''
            spatial_orb_str = ''
            for n in range(len(a_str)):
                spatial_orb_str += str(int(a_str[n]) + int(b_str[n]))
                spin_orb_str += a_str[n]
                spin_orb_str += b_str[n]

            if spatial_orb_str in det_dict.keys():
                det_dict[spatial_orb_str].append(ci[alpha_addr, beta_addr])
            else:
                det_dict[spatial_orb_str] = [ci[alpha_addr, beta_addr]]
            spin_orb_strings.append(spin_orb_str)
            coeff.append(ci[alpha_addr, beta_addr])
    print("Number of FCI determinants", len(det_dict.keys()))

    # Running CCSD
    print("\nCCSD")
    mycc = cc.CCSD(mf)
    result = mycc.kernel()
    t2 = result[2]
    t1 = result[1]
    ccsd_conv = result[0]


    # Spin Orbital HF configuration
    # hf_conf = '' 
    # for n in range(nelec):
    #     hf_conf += '1'

    # for m in range(norb*2-nelec):
    #     hf_conf += '0'
    # print('hf_conf', hf_conf)

    # Spatial Orbital HF configuration
    hf_conf = '' 

    for n in range(int(nelec/2)):
        hf_conf += '2'

    for m in range(norb - int(nelec/2)):
        hf_conf += '0'
    print('HF occupation number (spatial orbitals):', hf_conf)

    # Forming occupation number numpy array
    occ_array = np.zeros(norb)
    for n in range(norb):
        occ_array[n] = int(hf_conf[n])

    amp_dict = dict()
    det_vs_amp_ind = dict() # dictionary of t2 amplitudes corresponding to spatial orbital occupation number

    # double amplitudes
    strings_t2 = []
    for exc_from_i in range(np.shape(t2)[0]):
        for exc_from_j in range(np.shape(t2)[1]): 
            for exc_to_a in range(np.shape(t2)[2]):  
                for exc_to_b in range(np.shape(t2)[3]):
                    # print("ijab", 2*exc_from_i, 2*exc_from_j, 2*exc_to_a+nelec, 2*exc_to_b+nelec)
                    ini_occ = occ_array.copy()
                    ini_occ[exc_from_i] = ini_occ[exc_from_i] - 1
                    ini_occ[exc_from_j] = ini_occ[exc_from_j] - 1
                    ini_occ[exc_to_a + int(nelec/2)] = ini_occ[exc_to_a + int(nelec/2)] + 1
                    ini_occ[exc_to_b + int(nelec/2)] = ini_occ[exc_to_b + int(nelec/2)] + 1
                    string = ''
                    for l in range(np.shape(ini_occ)[0]):
                        string += str(int(ini_occ[l]))
                    if string in amp_dict.keys():
                        amp_dict[string].append(t2[exc_from_i, exc_from_j, exc_to_a, exc_to_b])
                        det_vs_amp_ind[string].append([exc_from_i, exc_from_j, exc_to_a, exc_to_b])
                    else:
                        amp_dict[string] = [t2[exc_from_i, exc_from_j, exc_to_a, exc_to_b]]
                        det_vs_amp_ind[string] = [[exc_from_i, exc_from_j, exc_to_a, exc_to_b]]
                    strings_t2.append(string)
                    
    print("Number of determinants formed from CCSD t2 amplitudes", len(strings_t2))


    det_vs_amp_t1_ind = dict() # dictionary of t1 amplitudes corresponding to spatial orbital occupation number
    # single amplitudes
    strings_t1 = []
    for exc_from_i in range(np.shape(t1)[0]):
        for exc_to_a in range(np.shape(t1)[1]):
            ini_occ = occ_array.copy()
            ini_occ[exc_from_i] = ini_occ[exc_from_i] - 1
            ini_occ[exc_to_a + int(nelec/2)] = ini_occ[exc_to_a + int(nelec/2)] + 1
            string = ''
            for l in range(np.shape(ini_occ)[0]):
                string += str(int(ini_occ[l]))
            if string in amp_dict.keys():
                amp_dict[string].append(t1[exc_from_i, exc_to_a])
                det_vs_amp_t1_ind[string].append([exc_from_i, exc_to_a])
            else:
                amp_dict[string] = [t1[exc_from_i, exc_to_a]]
                det_vs_amp_t1_ind[string] = [[exc_from_i, exc_to_a]]
            strings_t1.append(string)
#     print(det_vs_amp_t1_ind)
    print("num of CCSD t1 dets", len(strings_t1))
    # print(Counter(strings_t1).keys())
    # print(Counter(strings_t1).values())

    # MP2
    print("\nMP2")
    pt = mp.MP2(mf).run()
    mp2 = pt.run()
    mp2_dict = dict()

    # mp2 amplitudes
    strings_mp2 = []
    mp2_amps_to_keep = []
    amp_inds_smaller_than_thresh = []
    for exc_from_i in range(np.shape(mp2.t2)[0]):
        for exc_from_j in range(np.shape(mp2.t2)[1]): 
            for exc_to_a in range(np.shape(mp2.t2)[2]):  
                for exc_to_b in range(np.shape(mp2.t2)[3]):
                    if np.abs(mp2.t2[exc_from_i, exc_from_j, exc_to_a, exc_to_b]) < mp2_thresh:
                        amp_inds_smaller_than_thresh.append([exc_from_i, exc_from_j, exc_to_a, exc_to_b])
                    else:
                        mp2_amps_to_keep.append([exc_from_i, exc_from_j, exc_to_a, exc_to_b])
                    # print("ijab", 2*exc_from_i, 2*exc_from_j, 2*exc_to_a+nelec, 2*exc_to_b+nelec)
                    ini_occ = occ_array.copy()
                    ini_occ[exc_from_i] = ini_occ[exc_from_i] - 1
                    ini_occ[exc_from_j] = ini_occ[exc_from_j] - 1
                    ini_occ[exc_to_a + int(nelec/2)] = ini_occ[exc_to_a + int(nelec/2)] + 1
                    ini_occ[exc_to_b + int(nelec/2)] = ini_occ[exc_to_b + int(nelec/2)] + 1
                    string = ''
                    for l in range(np.shape(ini_occ)[0]):
                        string += str(int(ini_occ[l]))
                    # print("str", string)
                    if string in mp2_dict.keys():
                        mp2_dict[string].append(mp2.t2[exc_from_i, exc_from_j, exc_to_a, exc_to_b])
                    else:
                        mp2_dict[string] = [mp2.t2[exc_from_i, exc_from_j, exc_to_a, exc_to_b]]
                    strings_mp2.append(string)
                    
    print("number of MP2 t2 dets", len(strings_mp2))
    # print("MP2 amps smaller than {} is \n{}".format(mp2_thresh, amp_inds_smaller_than_thresh))
    # print(Counter(strings_mp2).keys())
    # print(Counter(strings_mp2).values())

    # Selected CI
    print("\nSelected CI")
    sci_solver = hci.SCI(mol)
    sci_solver.select_cutoff = sci_select_cutoff
    sci_solver.ci_coeff_cutoff = sci_ci_coeff_cutoff

    nmo = mf.mo_coeff.shape[1]
    h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
    h2 = ao2mo.full(mol, mf.mo_coeff)
    e_sci, civec = sci_solver.kernel(h1, h2, mf.mo_coeff.shape[1], mol.nelectron, ecore=mol.energy_nuc())
    print("e_sci", e_sci)
    print("Total number of selected CI determinants: {}".format(len(civec[0])))

    # For conversion of SCI to FCI vector for comparison
    nelec = int(nelec/2), int(nelec/2)
    neleca, nelecb = nelec

    # Transforming SCI vector to FCI format
    strsa = cistring.gen_strings4orblist(range(norb), neleca)
    stradic = dict(zip(strsa,range(strsa.__len__())))
    strsb = cistring.gen_strings4orblist(range(norb), nelecb)
    strbdic = dict(zip(strsb,range(strsb.__len__())))
    na = len(stradic)
    nb = len(strbdic)
    ndet = len(civec[0])
    fcivec = np.zeros((na,nb))
    for idet, (stra, strb) in enumerate(civec[0]._strs.reshape(ndet,2,-1)):
        ka = stradic[stra[0]]
        kb = strbdic[strb[0]]
        fcivec[ka,kb] = civec[0][idet] # SCI coefficients array in FCI format (with alpha and beta strings) 


    # Obtaining spatial orbital occupation numbers for SCI
    coeff_sci = list()
    spatial_orb_strings_sci = list()
    det_dict_sci = dict()
    for alpha_addr in range(fcivec.shape[0]):
        for beta_addr in range(fcivec.shape[0]):
            # In pyscf the lower energy orbitals are to the RIGHT!!! so we have to reverse the strings

            a_str = bin(cistring.addrs2str(norb, int(nelec[0]),[alpha_addr])[0])[2:].rjust(norb, '0')[::-1] # converting to strings with occ numbers
            b_str = bin(cistring.addrs2str(norb, int(nelec[1]),[beta_addr])[0])[2:].rjust(norb, '0')[::-1]
            spatial_orb_str = ''
            for n in range(len(a_str)):
                spatial_orb_str += str(int(a_str[n]) + int(b_str[n]))

            if spatial_orb_str in det_dict_sci.keys():
                det_dict_sci[spatial_orb_str].append(fcivec[alpha_addr, beta_addr])
            else:
                det_dict_sci[spatial_orb_str] = [fcivec[alpha_addr, beta_addr]]
            spatial_orb_strings_sci.append(spatial_orb_str)
            coeff_sci.append(fcivec[alpha_addr, beta_addr])


    filtered_sci_dets = filter_dets_from_dict(det_dict_sci, det_thresh)
    print("Number of SCI spatial orbitals configurations with CI coefficients > {} is {}".format(det_thresh, len(filtered_sci_dets.keys())))

    # Filter amps using SCI
    thrown_out_dets = []
    thrown_out_amps = []
    for key in det_dict_sci.keys():
        if key not in filtered_sci_dets.keys():
            thrown_out_dets.append(key) # this gives us list of dets that were filtered out
            # print(key, det_dict_sci[key])
            if key in amp_dict.keys():
                thrown_out_amps.append(np.abs(np.max(amp_dict[key]))) 

    # Creating list of t2 amplitudes to keep after SCI screening
    # matches amps with dets 
    amps_to_keep = []
    n_small_amps_kept = 0
    for key in filtered_sci_dets.keys():
    # for key in filtered_sci_dets.keys():
        if key in det_vs_amp_ind.keys():
            for amp in det_vs_amp_ind[key]:
                if np.abs(t2[amp[0], amp[1], amp[2], amp[3]]) > 1e-10:
                    # print(t2[amp[0], amp[1], amp[2], amp[3]]) 
                    amps_to_keep.append(amp)
                else:
                    # print(t2[amp[0], amp[1], amp[2], amp[3]]) 
                    n_small_amps_kept +=1 
        else:
            # print("key {}  is not in dict".format(key))
            pass

    # Creating list of t1 amplitudes to keep after SCI screening
    # matches amps with dets 
    amps_t1_to_keep = []
    n_small_t1_amps_kept = 0
    for key in filtered_sci_dets.keys():
    # for key in filtered_sci_dets.keys():
        if key in det_vs_amp_t1_ind.keys():
            for amp in det_vs_amp_t1_ind[key]:
                if np.abs(t1[amp[0], amp[1]]) > 1e-10:
                    amps_t1_to_keep.append(amp)
                else:
                    n_small_t1_amps_kept +=1 
        else:
            # print("key {}  is not in dict".format(key))
            pass
    # print("t1 amplitudes to keep:", amps_t1_to_keep)
    # print("t1 det vs amp", det_vs_amp_t1_ind)

    # Creating array for t2 amplitudes where we do prescreening using selected CI
    t2_filtered = np.zeros(np.shape(t2))
    for inds in amps_to_keep:
        t2_filtered[inds[0], inds[1], inds[2], inds[3]] = t2[inds[0], inds[1], inds[2], inds[3]]
        # print(t2[inds[0], inds[1], inds[2], inds[3]])

    # Creating array for t1 amplitudes where we do prescreening using selected CI
    t1_filtered = np.zeros(np.shape(t1))
    for inds in amps_t1_to_keep:
        t1_filtered[inds[0], inds[1]] = t1[inds[0], inds[1]]

    # filtering amps by MP2
    t2_filtered_by_mp2 = t2.copy()
    for inds in amp_inds_smaller_than_thresh:
        t2_filtered_by_mp2[inds[0], inds[1], inds[2], inds[3]] = 0

    # Results to save for analysis

    # Energies
    e_ccsd_mp2_filter = mycc.energy(t1, t2_filtered_by_mp2) + e_hf
    e_ccsd_sci_filter = mycc.energy(t1, t2_filtered) + e_hf
    # e_ccsd_sci_filter = mycc.energy(t1_filtered, t2_filtered) + e_hf # filtering t1 too
    e_ccsd = mycc.energy(t1, t2) + e_hf

    # Errors
    sci_err_wrt_fci = e_sci[0] - e_fci
    ccsd_err_wrt_fci = e_ccsd - e_fci
    err_wrt_ccsd = e_ccsd_sci_filter - e_ccsd
    err_wrt_fci = e_ccsd_sci_filter - e_fci
    err_ccsd_mp2_filter = e_ccsd_mp2_filter - e_fci

    # Amplitudes and determinants
    n_dets_fci = len(det_dict.keys())
    n_dets_sci = len(civec[0])
    n_ccsd_amp = np.shape(t2)[0]*np.shape(t2)[1]*np.shape(t2)[2]*np.shape(t2)[3]
    n_t2_amps_mp2_filter = n_ccsd_amp - len(amp_inds_smaller_than_thresh)
    n_amps_to_keep = len(amps_to_keep)
    thrown_out_array = np.asarray(thrown_out_amps)
    max_amp_thrown_out = np.max(thrown_out_array)
    n_amps_thrown_out = len(thrown_out_array[thrown_out_array>discarded_amp_thresh])

    print("\nResults:")

    print("{:<40s}{:>30.8e}".format("SCI Error WRT FCI:", sci_err_wrt_fci))
    print("{:<40s}{:>30.8e}".format("CCSD Error WRT FCI:", ccsd_err_wrt_fci))
    print("\n{:<40s}{:>30.8e}".format("Error WRT CCSD due to cutoff:", err_wrt_ccsd))
    print("{:<40s}{:>30.8e}".format("Error WRT FCI due to cutoff:", err_wrt_fci))

    print("\n{:<40s}{:>30.8e}".format("MP2 cutoff threshold", mp2_thresh))
    print("{:<40s}{:>30.8e}".format("Error of CCSD WRT FCI, MP2 cutoff ", err_ccsd_mp2_filter))
    print("{:<40s}{:>29d}".format("Number of CCSD t2 amplitudes, MP2 filter:", n_t2_amps_mp2_filter))


    print("\n{:<40s}{:>30d}".format("Total number of FCI determinants:", n_dets_fci))
    print("{:<40s}{:>30d}".format("Total number of SCI determinants:", n_dets_sci))
    print("\n{:<40s}{:>30d}".format("Number of CCSD t2 amplitudes", n_ccsd_amp))
    print("{:<40s}{:>30d}".format("Number of t2 amplitudes screened by SCI:", n_amps_to_keep))
    print("{:<30s}{:<2.2e}:{:>21d}".format("Number of kept amplitudes smaller than ", discarded_amp_thresh, n_small_amps_kept))
    print("{:<40s}{:<2.2e} thrown out:{:>10d}".format("Number of amplitudes larger than", discarded_amp_thresh,
        n_amps_thrown_out))
    print("\n{:<40s}{:>30.8e}".format("Largest discarded amplitude:", max_amp_thrown_out))
    
#     print("t2 amplitudes kept")
#     for amp in amps_to_keep:
#         print(amp, t2[amp[0], amp[1], amp[2], amp[3]])
    single_dets = filter_out_excitations_from_det_dict(det_dict, 1, mol.nelectron)
    double_dets = filter_out_excitations_from_det_dict(det_dict, 2, mol.nelectron)
    triple_dets = filter_out_excitations_from_det_dict(det_dict, 3, mol.nelectron)
    
    
    print("\nPrinting triply excited determinants")
    filtered_triples = filter_dets_from_dict(triple_dets, thresh=1e-04)
#     print_sorted_dict(filtered_triples)
    # print(occ_array)
    triples_dict = generate_triple_exc_dets(mol.nelectron, norb, occ_array)
    # print("triples_dict", triples_dict)
    print("Triple indices to include")
    t3_ind_to_include = []
    for key in filtered_triples.keys():
        # print("key", key)
        # print(triples_dict[key])
        for item in triples_dict[key]:
            t3_ind_to_include.append(item)

    print(nelec)
    print(norb)
    print("t1 before conversion", amps_t1_to_keep)
    sci_converted_t1 = convert_t1_to_blocks(amps_t1_to_keep, mol.nelectron, norb)
    print("CONVERTED", sci_converted_t1)

    print("\nt2 conversion")
    sci_converted_t2 = convert_t2_to_blocks(amps_to_keep, mol.nelectron, norb)  
    for n in range(len(amps_to_keep)):
        print(amps_to_keep[n], "--->", sci_converted_t2[n], t2[amps_to_keep[n][0], amps_to_keep[n][1], amps_to_keep[n][2], amps_to_keep[n][3]])

    # Converting MP2 amps to from t2 array indices into block spin format
    mp2_converted_t2 = convert_t2_to_blocks(mp2_amps_to_keep, mol.nelectron, norb) 
    
    # print("TEST")
    # t2_filtered[1, 2, 2, 3] = 0
    # e_test = mycc.energy(t1, t2_filtered) + e_hf
    # print(e_ccsd_sci_filter - e_test) 
#     print("t3 before conversion\n", t3_ind_to_include)
#     print("\nCONVERTED\n", convert_t3_to_blocks(t3_ind_to_include, mol.nelectron, norb))
    return sci_converted_t2, mp2_converted_t2

In [131]:
def generate_t2_indices(norb, nelec):
    # generates single excitations indices for RHF wave function
    inds = []
    for exc_from_i in range(int(nelec/2)):
        for exc_from_j in range(int(nelec/2)):
            for exc_to_a in range(norb-int(nelec/2)):
                for exc_to_b in range(norb-int(nelec/2)):
                    inds.append([exc_from_i, exc_to_a+int(nelec/2), exc_from_j+norb, exc_to_b+norb+int(nelec/2)])
    return inds

def generate_t1_indices(norb, nelec, ini_conf=None):
    # generates single excitations indices for RHF wave function if ini_conf is None
    if ini_conf is None:
        occ = np.zeros(norb*2, ndtype=int)
        for n in range(int(nelec/2)):
            alpha_occ[n] = 1
            beta_occ[n + norb] = 1
    inds = []
    amp_vs_det_dict = dict()
    for exc_from_i in range(int(nelec/2)):
        for exc_to_a in range(norb - int(nelec/2)):
            inds.append([exc_from_i, exc_to_a + int(nelec/2)])
            inds.append([exc_from_i + norb, exc_to_a + norb + int(nelec/2)])
    print(alpha_occ, beta_occ)
    return inds

In [117]:
r = 1.5
basis = 'sto-3g'

# For H2O molecule
theta = 104.5
c,s = np.cos(np.radians(theta)), np.sin(np.radians(theta))
R = np.array(((c, -s), (s, c))) #Rotation matrix
x,y = R @ np.array((0, r))

atom='O .0 .0 .0; H .0 .0 {}; H .0 {} {}'.format(r, x, y)
# atom = 'Be .0 .0 .0; H .0 .0 {}; H .0 .0 {}'.format(r, -r)
# atom = 'H 0 0 0; H 0 0 {}; H 0 0 {}; H 0 0 {}'.format(r, r*2, r*3)
# atom='Li 0 0 0; H 0 0 {}'.format(r)
# atom='N 0 0 0; N 0 0 {}'.format(r)
# atom='H 0 0 0; H 0 0 {}; H 0 0 {}; H 0 0 {}; H 0 0 {}; H 0 0 {}'.format(r, r*2, r*3, r*4, r*5)

# For UCCSD calculations
driver = PySCFDriver(atom=atom, unit=UnitsType.ANGSTROM,
                     charge=0, spin=0, basis=basis)
molecule = driver.run()
e_nr = molecule.nuclear_repulsion_energy
core = Hamiltonian(transformation=TransformationType.FULL, qubit_mapping=QubitMappingType.JORDAN_WIGNER,
                   two_qubit_reduction=False, freeze_core=False)
qubit_op, aux_ops = core.run(molecule)
print("UCCSD requires {} qubits".format(qubit_op.num_qubits))


# For pure PySCF calculations
mol = gto.M(atom=atom, basis=basis)
norb_spatial = int(qubit_op.num_qubits/2)
nelec = molecule.num_alpha + molecule.num_beta

sci_select_cutoff = .001
sci_ci_coeff_cutoff = .001
det_thresh = 1e-15 # filters SCI selected dets (in FCI format there's a lot of zeros)
discarded_amp_thresh = 1e-06 # for counting amplitudes filtered out by SCI
mp2_thresh = 1e-6 # throw out mp amps smaller than this 

# Running PySCF calculations for filtering amplitudes
t2_sci_filtered, t2_mp2_filtered = run_calcs(r, mol, basis, sci_select_cutoff, sci_ci_coeff_cutoff, det_thresh, discarded_amp_thresh, mp2_thresh)

UCCSD requires 14 qubits
r= 1.5

FCI
FCI energy -74.87343608825451
nelec 10
norb 7
Num Strings:  21
Number of FCI determinants 161

CCSD
HF occupation number (spatial orbitals): 2222200
Number of determinants formed from CCSD t2 amplitudes 100
num of CCSD t1 dets 10

MP2
number of MP2 t2 dets 100

Selected CI
e_sci [-74.8734341]
Total number of selected CI determinants: 105
Number of SCI spatial orbitals configurations with CI coefficients > 1e-15 is 52

Results:
SCI Error WRT FCI:                                      1.98642088e-06
CCSD Error WRT FCI:                                     9.59937710e-04

Error WRT CCSD due to cutoff:                           0.00000000e+00
Error WRT FCI due to cutoff:                            9.59937710e-04

MP2 cutoff threshold                                    1.00000000e-06
Error of CCSD WRT FCI, MP2 cutoff                       9.59937710e-04
Number of CCSD t2 amplitudes, MP2 filter:                           34

Total number of FCI determinants

In [118]:
# Inspecting the 2e integrals array
h1 = molecule.one_body_integrals
h2 = molecule.two_body_integrals
print("shape of 2e- integrals array", np.shape(h2))

# int_vals = []
# converted_ind = []
# filtered_ind = np.argwhere(h2 > 1e-8)
# for ind in filtered_ind:
#     # ijkl -> ljik
#     converted_ind.append([ind[3], ind[1], ind[0], ind[2]])
#     int_vals.append(h2[ind[0], ind[1], ind[2], ind[3]])
# ind_amp = sorted(zip(converted_ind, int_vals), key=lambda kv: np.abs(kv[1]), reverse=True)

# n_total = 20
# for n in range(n_total):
#     print(ind_amp[n])
# print(len(filtered_ind))

shape of 2e- integrals array (14, 14, 14, 14)


In [132]:
# Filtering t1 amplitudes
t1_inds = generate_t1_indices(norb_spatial, nelec)
filtered_t1 = []
filtered_t1_amps = []
thresh = 1e-08
for t1 in t1_inds:
    if np.abs(h1[t1[0], t1[1]]) > thresh:
        filtered_t1.append(t1)
        filtered_t1_amps.append(h1[t1[0], t1[1]])
print(filtered_t1)

TypeError: 'ndtype' is an invalid keyword argument for zeros()

The following computations require the integrals stored in the '*chemist*' notation

            h2(i,j,k,l) --> adag_i adag_k a_l a_j

       and the integral values are used for the coefficients of the second-quantized
       Hamiltonian that is built. The integrals input here should be in block spin
       format and also have indexes reordered as follows 'ijkl->ljik'""""

In [120]:
# Filtering amplitudes which are connected by 2-electron integrals elements larger than threshold
t2_inds = generate_t2_indices(norb_spatial, nelec)
print("Total number of amplitudes:", len(t2_inds))
filtered_t2 = []
filtered_t2_amps = []
thresh = 1e-8
for t2 in t2_inds:
    if np.abs(h2[t2[3], t2[1], t2[0], t2[2]]) > thresh:
        filtered_t2.append(t2)
        filtered_t2_amps.append(h2[t2[3], t2[1], t2[0], t2[2]])
        sorted_filtered_t2 = sorted(zip(filtered_t2, filtered_t2_amps), key=lambda kv: np.abs(kv[1]), reverse=True)
print("Number of t2 amplitudes filtered by 2e integrals amplitudes", len(filtered_t2))
print("Number of t2 amplitudes filtered by SCI", len(t2_sci_filtered))
print("Number of t2 amplitudes filtered by MP2", len(t2_mp2_filtered))


# Printing 2e integrals values for matching indices with amps filtered by SCI
n_matches_with_SCI_sel = 0
for ind_2e_int_sel in t2_inds: # selected by 2e integrals
    for ind_sci_sel in t2_sci_filtered: # selected by SCI
        if ind_2e_int_sel == ind_sci_sel:
            n_matches_with_SCI_sel += 1
#             print(ind_2e_int_sel, h2[ind_2e_int_sel[3], ind_2e_int_sel[1], ind_2e_int_sel[0], ind_2e_int_sel[2]])
print("Number of matches of 2e filtering with SCI filtering is {}".format(n_matches_with_SCI_sel))


# Printing 2e integrals values for matching indices with amps filtered by MP2
print("Matching with MP2")
n_matches_with_mp2_sel = 0
for ind_2e_int_sel in filtered_t2: # selected by 2e integrals
    for ind_mp2_sel in t2_mp2_filtered: # selected by SCI
        if ind_2e_int_sel == ind_mp2_sel:
            n_matches_with_mp2_sel += 1
#             print(ind_2e_int_sel, h2[ind_2e_int_sel[3], ind_2e_int_sel[1], ind_2e_int_sel[0], ind_2e_int_sel[2]])
print("Number of matches of 2e filtering with MP2 filtering is {}".format(n_matches_with_mp2_sel))


Total number of amplitudes: 100
Number of t2 amplitudes filtered by 2e integrals amplitudes 34
Number of t2 amplitudes filtered by SCI 34
Number of t2 amplitudes filtered by MP2 34
Number of matches of 2e filtering with SCI filtering is 34
Matching with MP2
Number of matches of 2e filtering with MP2 filtering is 34


In [110]:
int_vals = []
filtered_ind = np.argwhere(h1 > 1e-8)
for ind in filtered_ind:
    int_vals.append(h1[ind[0], ind[1]])
ind_amp = sorted(zip(filtered_ind, int_vals), key=lambda kv: np.abs(kv[1]), reverse=True)
for n in range(len(int_vals)):
    print(ind_amp[n])

(array([0, 1]), 0.618957003912905)
(array([7, 8]), 0.618957003912905)
(array([1, 0]), 0.6189570039129028)
(array([8, 7]), 0.6189570039129028)
(array([1, 5]), 0.14439219533843292)
(array([5, 1]), 0.14439219533843292)
(array([ 8, 12]), 0.14439219533843292)
(array([12,  8]), 0.14439219533843292)
(array([1, 3]), 0.020329095440354675)
(array([3, 1]), 0.020329095440354675)
(array([ 8, 10]), 0.020329095440354675)
(array([10,  8]), 0.020329095440354675)
(array([0, 3]), 0.005576214403144745)
(array([ 7, 10]), 0.005576214403144745)
(array([3, 0]), 0.005576214403144736)
(array([10,  7]), 0.005576214403144736)
(array([4, 3]), 1.2692553895089699e-08)
(array([11, 10]), 1.2692553895089699e-08)
(array([3, 4]), 1.2692553835961488e-08)
(array([10, 11]), 1.2692553835961488e-08)


In [111]:
fer_op = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)

In [112]:
ee = NumPyMinimumEigensolver(qubit_op)
result = core.process_algorithm_result(ee.run())
exact_e = result['computed_electronic_energy']
print("exact_e", exact_e)

exact_e -78.26118209697104


In [113]:
init_state = HartreeFock(num_orbitals=core._molecule_info['num_orbitals'],
                    qubit_mapping=core._qubit_mapping, two_qubit_reduction=core._two_qubit_reduction,
                    num_particles=core._molecule_info['num_particles'])

In [114]:
var_form = UCCSD(num_orbitals=core._molecule_info['num_orbitals'],
                   num_particles=core._molecule_info['num_particles'],
                   active_occupied=None, active_unoccupied=None, initial_state=init_state,
                   qubit_mapping=core._qubit_mapping, two_qubit_reduction=core._two_qubit_reduction,
                   num_time_slices=1,
                   single_exc_op_list=[[1, 2], [7, 8], [0, 2], [6, 8], [1, 5], [7, 11], [0, 5], [6, 11]],
                   double_exc_op_list=[[0, 2, 7, 11], [0, 5, 7, 8], [1, 2, 6, 11], [1, 5, 6, 8], [1, 2, 7, 8], [0, 2, 7, 8], [1, 2, 6, 8],
                   [1, 2, 7, 11], [1, 5, 7, 8], [0, 2, 6, 8], [0, 2, 6, 11], [0, 5, 6, 8], [1, 3, 7, 9], [0, 3, 7, 9], 
                   [1, 3, 6, 9], [0, 3, 6, 9], [1, 4, 7, 10], [0, 4, 7, 10], [1, 4, 6, 10], [0, 4, 6, 10], [1, 5, 7, 11], [0, 5, 7, 11], [1, 5, 6, 11], [0, 5, 6, 11]],
                   triple_exc_op_list=[[0, 2, 7, 8, 7, 11], [0, 2, 7, 11, 7, 8], [0, 5, 7, 8, 7, 8], [1, 2, 6, 8, 7, 11], [1, 2, 6, 11, 7, 8], [1, 5, 6, 8, 7, 8]],
                )

vqe = VQE(qubit_op,  var_form=var_form, initial_point=None) # particle number and S^2 expectation values


In [ ]:
print(len(var_form._triple_excitations), len(var_form._double_excitations), len(var_form._single_excitations))
# print(var_form._hopping_ops)
# for op in var_form._hopping_ops:
#     print(op)
n=35
# print(var_form._hopping_ops[n].evolve().decompose())
print(len(var_form._hopping_ops[n]._paulis))

In [ ]:
backend_type = 'statevector_simulator'
backend = Aer.get_backend(backend_type)
quantum_instance = QuantumInstance(backend=backend, shots=10000)
ret = vqe.run(quantum_instance)
# print('computed e', ret['optimal_value'])
# print("\nResults:")
# print(ret)

In [ ]:
n = 10
print(qubit_op.paulis[n][1])
print(qubit_op.paulis[n][0])
print(qubit_op.paulis[n][1].z)
print(qubit_op.paulis[n][1].x)
print(qubit_op.paulis[n][1].phase)